Brian speaks in terms of our libraries and the DCC speaks in terms of when records are created at their site.

Lets compare the to help communicate.

In [1]:
import pandas
import numpy
import os
import datetime
import pprint
import django

In [2]:
from curation_common import *

In [3]:
if not 'DJANGO_SETTINGS_MODULE' in os.environ:
    os.environ['DJANGO_SETTINGS_MODULE'] = 'htsworkflow.settings.myrada'
django.setup()

In [4]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [5]:
dcc_query = server.get_json('https://www.encodeproject.org/search/?type=experiment&lab.title=Barbara%20Wold,%20Caltech&award.rfa=ENCODE2')

In [6]:
import experiments.models
import samples.models

In [11]:
found = []

for minimal in dcc_query['@graph']:
    exp_id = minimal['@id']
    experiment = server.get_json(exp_id)
    for rep in experiment['replicates']:
        library = rep['library']
        aliases = library['aliases']
        if len(aliases) == 0:
            print("No alias for {}".format(library['@id']))
        elif len(aliases) > 1:
            print("Too many aliases for {}. [{}]".format(
                    library['@id'], ','.join(library['aliases'])))
        else:
            jumpgate_id = aliases[0].replace('barbara-wold:', '')
            date_submitted = datetime.datetime.strptime(library['date_created'][:10], '%Y-%m-%d')
            try:
                jumpgate = samples.models.Library.objects.get(pk=jumpgate_id)
                found.append({
                        'experiment': exp_id,
                        'library': library['@id'],
                        'jumpgate': jumpgate_id,
                        'submitted_date': date_submitted.date(),
                        'creation_date': jumpgate.creation_date,
                    }
                )
            except samples.models.Library.DoesNotExist as e:
                print("Library {} not found".format(jumpgate_id))

report = pandas.DataFrame(found, columns=['experiment', 'library', 'jumpgate', 'submitted_date', 'creation_date'])

Library SL2970 not found
Library SL2971 not found


In [12]:
report.sort(['jumpgate'])

,experiment,library,jumpgate,submitted_date,creation_date
30,/experiments/ENCSR000CWL/,/libraries/ENCLB661EBH/,02973,2013-12-31,None
40,/experiments/ENCSR000EYN/,/libraries/ENCLB150CGC/,10515,2013-12-31,2008-11-10
12,/experiments/ENCSR077AZT/,/libraries/ENCLB858EYY/,10516,2014-02-11,2008-11-10
44,/experiments/ENCSR000EYO/,/libraries/ENCLB296STQ/,10517,2013-12-31,2008-11-10
39,/experiments/ENCSR000EYN/,/libraries/ENCLB700LMU/,10565,2013-12-31,2008-12-19
43,/experiments/ENCSR000EYO/,/libraries/ENCLB912OTZ/,10567,2013-12-31,2008-12-19
21,/experiments/ENCSR000EYP/,/libraries/ENCLB211VHB/,10874,2013-12-31,2009-05-27
9,/experiments/ENCSR962TBJ/,/libraries/ENCLB827ADT/,10876,2014-02-11,2009-05-27
41,/experiments/ENCSR000EYR/,/libraries/ENCLB596OCH/,10878,2013-12-31,2009-05-27
42,/experiments/ENCSR000EYR/,/libraries/ENCLB285RFV/,10879,2013-12-31,2009-05-27


In [14]:
report.to_csv('/tmp/date_submitted_and_created.csv', index=False)